In [33]:
# importanto bibliotecas
import pandas as pd
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt

In [34]:
# conexão redshift 
connection = psycopg2.connect(dbname="dw_amorsaude",

                  user="lucasmateus",

                  password="df93aS@aK0sd",

                  host ="amorsaude-cluster-redshift.cmc4ev2oyczd.us-east-1.redshift.amazonaws.com",

                  port = 5439)

In [35]:
# setando dataframe a partir do arquivo de excel e extraindo para uma lista
excel_path = '/home/lucasmateus/dbt-projects/projetos_lucas/enderecos_pacientes_amei/pacientes Amei Sem Endereço.xlsx'
df_excel = pd.read_excel(excel_path)
cpf_list = df_excel['cpf'].tolist()

In [36]:
# setando a cláusula WHERE da query
cpf_placeholders = ', '.join(['%s'] * len(cpf_list))

# definindo query
query = f"""
select
	sp.cpf,
	sp.nome_paciente,
	spe.cep,
	spe.cidade,
	spe.estado,
	spe.bairro,
	spe.logradouro,
	spe.numero,
	spe.complemento
from stg_pacientes sp 
left join stg_paciente_endereco spe on spe.id = sp.id
where sp.cpf IN ({cpf_placeholders})"""

In [37]:
# setando o dataframe com despesas
df = pd.read_sql_query(query,con=connection, params=cpf_list)

/tmp/ipykernel_646/503454361.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,con=connection, params=cpf_list)


In [38]:
df.to_excel('bd_endereco_feegow.xlsx', index=False)